In [35]:
# Importation des bibliothèques nécessaires
from keras.preprocessing.image import img_to_array  # Convertir une image en tableau NumPy
from sklearn.utils import shuffle  # Mélanger les données
from sklearn.preprocessing import LabelBinarizer  # Convertir les étiquettes en format binaire
from sklearn.model_selection import train_test_split  # Diviser les données en ensembles d'entraînement et de test
from keras.preprocessing.image import ImageDataGenerator  # Génération d'images augmentées
from sklearn.metrics import classification_report, confusion_matrix  # Évaluation des performances du modèle
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img  # Traitement d'images avec Keras


In [2]:
train_data_dir = '/kaggle/input/agriculture-crop-images/kag2' 

**Data Augmentation Configuration and Training Data Generator Initialization**


In [3]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0, horizontal_flip=True,vertical_flip=True, rotation_range=90)
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(224,224),
        batch_size=64,
        class_mode='categorical',
        shuffle=True)

Found 804 images belonging to 5 classes.


In [4]:
train_generator.class_indices


{'jute': 0, 'maize': 1, 'rice': 2, 'sugarcane': 3, 'wheat': 4}

**Model Training InceptionV3**


In [15]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model

**Initialization of InceptionV3 Pre-trained Model and Freezing Layers**


In [16]:
pre_model = InceptionV3(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (224, 224, 3))

for layer in pre_model.layers:
    layer.trainable = False  

87910968/87910968 [==============================] - 4s 0us/step


In [17]:
pre_model.summary()



Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 111, 111, 32)         0         ['batch_normalizati

**Adding Flatten and Final Dense Layer to InceptionV3 Pre-trained Model**

In [18]:
last_layer = Flatten()(pre_model.output)
final_layer = Dense(5, activation='softmax')(last_layer)

In [19]:
model = Model(inputs=pre_model.input, outputs=final_layer)


In [20]:
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 111, 111, 32)         0         ['batch_normalization[0]

**Compiling the Model**

In [21]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')


In [22]:
model.fit_generator(train_generator, epochs=10)


/tmp/ipykernel_42/2975305634.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10)


Epoch 1/10
13/13 [==============================] - 55s 4s/step - loss: 14.0672 - accuracy: 0.2724
Epoch 2/10
13/13 [==============================] - 49s 4s/step - loss: 4.6594 - accuracy: 0.5087
Epoch 3/10
13/13 [==============================] - 50s 4s/step - loss: 1.6313 - accuracy: 0.6803
Epoch 4/10
13/13 [==============================] - 49s 4s/step - loss: 1.0570 - accuracy: 0.7612
Epoch 5/10
13/13 [==============================] - 51s 4s/step - loss: 0.7754 - accuracy: 0.8072
Epoch 6/10
13/13 [==============================] - 50s 4s/step - loss: 0.4986 - accuracy: 0.8470
Epoch 7/10
13/13 [==============================] - 50s 4s/step - loss: 0.5066 - accuracy: 0.8495
Epoch 8/10
13/13 [==============================] - 49s 4s/step - loss: 0.5114 - accuracy: 0.8396
Epoch 9/10
13/13 [==============================] - 50s 4s/step - loss: 0.4587 - accuracy: 0.8694
Epoch 10/10
13/13 [==============================] - 49s 4s/step - loss: 0.3238 - accuracy: 0.8930


**Save Model**


In [23]:
model.save('my_model.keras')

In [24]:
import matplotlib.pyplot as plt


**predictive system**


In [25]:
#TestData
test_data_dir = '/kaggle/input/agriculture-crop-images/crop_images'
test_datagen = ImageDataGenerator(rescale=1./255)
 
test_batchsize = 64
 
test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(224,225),
        batch_size=64,
        class_mode='categorical',
        shuffle=False)

Found 201 images belonging to 5 classes.


In [26]:
import numpy as np
model.load_weights("my_model.keras")
class_labels = test_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())
Y_pred = model.predict_generator(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

/tmp/ipykernel_42/119025590.py:6: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(test_generator)


In [27]:
import cv2
import numpy as np

**Some predictions**


In [28]:
def predict_crop(path,actual,class_labels):
    predict_datagen = ImageDataGenerator(rescale=1./255)
    img = cv2.imread(path)
    img = cv2.resize(img, (224, 224))
    img = np.array(img).reshape((1, 224, 224, 3))
    Y_pred = model.predict(img)
    y_pred = np.argmax(Y_pred, axis=1)
    if y_pred == actual:
        print('Correct prediction')
    else:
        print("Wrong Prediction!!")
    print('Actual class "{0}" and predicted class "{1}"'.format(class_labels[int(y_pred)],class_labels[actual]))

In [29]:
predict_crop('/kaggle/input/agriculture-crop-images/test_crop_image/jute-field.jpg',0,class_labels)

1/1 [==============================] - 2s 2s/step
Wrong Prediction!!
Actual class "rice" and predicted class "jute"


In [30]:
predict_crop('/kaggle/input/agriculture-crop-images/test_crop_image/maize_fieldmexico.jpeg',1,class_labels)

1/1 [==============================] - 0s 126ms/step
Wrong Prediction!!
Actual class "sugarcane" and predicted class "maize"


In [31]:
predict_crop('/kaggle/input/agriculture-crop-images/test_crop_image/rice-field.jpg',0,class_labels)

1/1 [==============================] - 0s 93ms/step
Wrong Prediction!!
Actual class "rice" and predicted class "jute"


In [32]:
predict_crop('/kaggle/input/agriculture-crop-images/test_crop_image/sugarcane fields.jpg',1,class_labels)

1/1 [==============================] - 0s 100ms/step
Wrong Prediction!!
Actual class "rice" and predicted class "maize"


**Evaluating the model**

In [34]:
model.evaluate_generator(test_generator)

/tmp/ipykernel_42/2205837447.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(test_generator)


[0.7234866619110107, 0.7661691308021545]